<a href="https://colab.research.google.com/github/arnold402/NLP_project/blob/main/Alina%2C_your_new_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Meet Alina, your new assistant


A NLP project for DSI Africa 2022 module 3 by Antsa, Arnold and Ulrich, tutored by Edoardo.

Why would you miss your important meeting/seminars/lecture if you can be updated on all events of your DSI calendar by asking a chatbot ?

Alina is a simple chatbot that interacts with google calendar. The code consists of 3 main parts: a chatbot section, an nlp section and a google calendar query API section.

The chatbot setion using google speech recogniton to listen the to users input and also to speak to the user (by converting text to speech). The chatbot currently supports English and French. The chatbot, which is called Alina, can be awakened using any of the following phrases "Hi Alina", "Hey Alina" or "Bonjour Alina" (for french). Based on the wake-up phrase, we choose the chatbot sets the default language it expects to be either English or French. The chatbot can ended be using any phrase that involves "bye" or "Au Revoir".

The chatbot listens to any user and converts any input phrase to a text (string). The spoken text is then passed to the machine learning model, which tries to identify what is the user's query. For this purpose, we used a multilingual model pre-trained on 15 languages to encode the input sentence from the user. This sentence is then compared against our database's English and French encodings of various sentences. Each of these sentences represents a query that the assistant can perform. We use a cosine distance as a similarity metric between the new sentence encoding and all the sentences in our database. The query for the sentence with the maximum distance is selected. We also set a threshold only to validate the query if the maximum distance is at aleast 0.5 (as the cosine similarity ranges from 0 to 1).

Finally, the selected query is implemented using functionalities from google calendar API. The API requires the user to provide a credentials file to allow the app to access their google calendar. Furthermore, a little bit of string manipulations and regular expressions (handle by the dateutil package) is required to capture dates and time information from the input sentence. The queried information from the google calendar is then passed to the text-to-speech functionality of the chatbot to respond to the user.

The chatbot has two different version, the one running in a local machine which can be implemented via following the read.me file of this repository and a one that is running in Colab Notebook, which you are currently reading.

Some changes were made to adapt to the fact that the speach recognition for the chatbot itself could not access the local microphone of our computer.


The chatbot was inspired by [this Medium article](https://towardsdatascience.com/ai-chatbot-with-nlp-speech-recognition-transformers-583716a299e9), the Google API codes from [this tutorial](https://www.codespeedy.com/access-google-calendar-data-with-python/). 


To access the microphone, we used code from [this notebook](https://colab.research.google.com/github/snakers4/silero-models/blob/master/examples.ipynb#scrollTo=jc7ZqfooYZnD).

### Getting ready...let us prepare ourself

First of all, I will be mounting my google drive where I stored my credential for my calendar.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/Calendar

/content/gdrive/MyDrive/Calendar


Then let us install the requirements. 

In [ ]:
# !pip install google-api-core==2.5.0
# !pip install google-api-python-client==1.12.10
# !pip install google-auth==2.3.3
# !pip install google-auth-httplib2==0.1.0
# !pip install google-auth-oauthlib==0.4.6
# !pip install google-images-download==2.8.0
# !pip install google-pasta==0.2.0
# !pip install googleapis-common-protos==1.54.0
# !pip install sentence-transformers
# !pip install SpeechRecognition
# !pip install gTTS
# !pip install transformers
# !apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
# !pip install PyAudio
# !pip install fasttext

Now, we are going to get the pretrained model we will be using.

In [3]:
# !wget -P /content/gdrive/MyDrive/Calendar/model/ https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin


--2022-04-11 22:35:08--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/content/gdrive/MyDrive/Calendar/model/lid.176.bin’

lid.176.bin         100%[===================>] 125.18M  36.5MB/s    in 4.0s    

2022-04-11 22:35:13 (31.1 MB/s) - ‘/content/gdrive/MyDrive/Calendar/model/lid.176.bin’ saved [131266198/131266198]



In [4]:
ls  model/

lid.176.bin


Now that we have all our requirements and downloaded the model, we are ready to go.

### dd